In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys


In [ ]:
train_path = "/content/drive/MyDrive/Subinay/statute_prediction/train3.csv"

In [ ]:
df = pd.read_csv(train_path)
df=df.drop(['name'], axis=1)
#df=df[(df['300']!=0)|(df['302']!=0)|(df['304']!=0)|(df['306']!=0)|(df['307']!=0)]
print(len(df))
train_df=df.iloc[0:408]
val_df=df.iloc[250:255]
#test_df=df.iloc[5:6]

409


In [ ]:
target_list = ['148','300','302','304','307']

In [ ]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05

In [ ]:
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer, AutoModel


In [ ]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained("law-ai/InLegalBERT")

In [ ]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['text']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [ ]:
# train_size = 0.8
# train_df = train_df.sample(frac=train_size, random_state=200).reset_index(drop=True)
# val_df = train_df.drop(train_df.index).reset_index(drop=True)


In [ ]:
print(len(val_df))

5


In [ ]:
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=2
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)


In [ ]:
print(len(val_data_loader))

4


In [ ]:


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')



In [ ]:
device

device(type='cpu')

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        #self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.bert_model = AutoModel.from_pretrained("law-ai/InLegalBERT")
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 5)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at law-ai/InLegalBERT were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)


In [ ]:
val_targets=[]
val_outputs=[]

In [ ]:
def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path):

  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf


  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        #if batch_idx%5000==0:
         #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)

    print('############# Epoch {}: Training End     #############'.format(epoch))
    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [ ]:
ckpt_path = "/content/drive/MyDrive/Subinay/statute_prediction/curr_ckpt"
best_model_path = "/content/drive/MyDrive/Subinay/statute_prediction/best_model.pt"

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1  Done   #############



In [ ]:
# example = df.iloc[74]['text']
# #example="Someone should be hanged for crime"
# print(example)
# encodings = tokenizer.encode_plus(example,None,add_special_tokens=True,max_length=MAX_LEN, padding='max_length',return_token_type_ids=True,truncation=True,return_attention_mask=True,return_tensors='pt')
# model.eval()
# with torch.no_grad():
#     input_ids = encodings['input_ids'].to(device, dtype=torch.long)
#     attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
#     token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
#     output = model(input_ids, attention_mask, token_type_ids)
#     final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
#     print(final_output)
#     print(train_df.columns[1:].to_list())

In [ ]:
def test(text):
  encodings = tokenizer.encode_plus(text,None,add_special_tokens=True,max_length=MAX_LEN, padding='max_length',return_token_type_ids=True,truncation=True,return_attention_mask=True,return_tensors='pt')
  model.eval()
  with torch.no_grad():
    input_ids = encodings['input_ids'].to(device, dtype=torch.long)
    attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
    token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
    output = model(input_ids, attention_mask, token_type_ids)
    final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
    print(final_output)
    print(train_df.columns[1:].to_list())


In [ ]:
test_path = "/content/drive/MyDrive/Subinay/statute_prediction/test3_label_gpt3.5_riot_evidence.csv"

In [ ]:
df = pd.read_csv(test_path)
df=df.drop(['name'], axis=1)
#df=df[(df['300']!=0)|(df['302']!=0)|(df['304']!=0)|(df['306']!=0)|(df['307']!=0)]
print(len(df))
test_df=df.iloc[0:600]
test_df

600


,text,appended_text,148,300,302,304,307
0,of 1999 with of 1999 was as follows a stage sh...,of 1999 with of 1999 was as follows a stage sh...,1,1,1,1,1
1,saddik lalo and others v state of of 3 crimina...,saddik lalo and others v state of of 3 crimina...,1,1,1,1,0
2,criminal appeal nos of 2004 the judgment was d...,criminal appeal nos of 2004 the judgment was d...,1,1,1,1,0
3,supreme court of india 1 appeal crl of the jud...,supreme court of india 1 appeal crl of the jud...,1,1,1,1,0
4,v state of of 24 criminal appeal no 1693 of no...,v state of of 24 criminal appeal no 1693 of no...,1,1,1,1,0
...,...,...,...,...,...,...,...
595,v state of of 3 appeal crl of 2004 the judgmen...,v state of of 3 appeal crl of 2004 the judgmen...,0,0,1,1,0
596,the appeal preferred by the appellant in that ...,the appeal preferred by the appellant in that ...,1,0,1,0,0
597,shankaraya naik and others v state of of 22 cr...,shankaraya naik and others v state of of 22 cr...,1,0,1,0,0
598,sarman and others v state of of india 7 of was...,sarman and others v state of of india 7 of was...,0,0,1,1,0


In [ ]:
for i in range(len(test_df)):
  text=test_df.iloc[i]['appended_text']
  test(text)

[[0.37448742985725403, 0.42692896723747253, 0.39765915274620056, 0.46304139494895935, 0.3818172812461853]]
['148', '300', '302', '304', '307']
[[0.3795301616191864, 0.4256438612937927, 0.4019148051738739, 0.47711852192878723, 0.3833736479282379]]
['148', '300', '302', '304', '307']
[[0.3740510642528534, 0.4267752766609192, 0.3995239734649658, 0.45557701587677, 0.3791613280773163]]
['148', '300', '302', '304', '307']
[[0.35906925797462463, 0.4173322916030884, 0.39914482831954956, 0.4620074927806854, 0.37207651138305664]]
['148', '300', '302', '304', '307']
[[0.3715081214904785, 0.42229774594306946, 0.40649107098579407, 0.4501417577266693, 0.3806477189064026]]
['148', '300', '302', '304', '307']
[[0.38059714436531067, 0.42711132764816284, 0.41169968247413635, 0.4486270546913147, 0.38606569170951843]]
['148', '300', '302', '304', '307']
[[0.3631356656551361, 0.41529056429862976, 0.40596187114715576, 0.4673827588558197, 0.3734539747238159]]
['148', '300', '302', '304', '307']
[[0.379347622